# Establishing a connection with Sql Server

In [379]:
import pyodbc

try:
    con=pyodbc.connect(
        'DRIVER={Odbc Driver 17 for SQL Server};'
        'SERVER=DESKTOP-4IVU8N4\\MSSQLSERVER1;'
        'UID=sa;'
        'PWD=abcd;',
        autocommit=True
    )
except Exception as e:
        print("Connect it properly",e)

# Creation of Database using Sql Server

In [ ]:
try:
    cursor=con.cursor()
    demo = 'SCD'
    create_db_query=f"Create database {demo};"
    cursor.execute(create_db_query)
    print("Hoorah, Database Created!!")
except Exception as e:
    print('Work more on Database Connection')

Work more on Database Connection


# Using SQLITE3, it is used to create a database within the venv for temporary purpose


In [381]:
# import sqlite3

# try:
#     con=sqlite3.connect("SCD.db")
#     print("connected successfully")
# except Exception as e:
#     print("not connected")

# Connecting with Created Database

In [382]:
from sqlalchemy import create_engine

try:
    engine = create_engine('mssql+pyodbc://@DESKTOP-4IVU8N4\MSSQLSERVER1/SCD?driver=Odbc+Driver+17+for+SQL+Server&trusted_connection=yes')
    print("Man, you made it!")
except Exception as e:
    print("Database not connected!, Think more you are too close")

Man, you made it!


# SCD-0

## Inserting the Values through hard coded method

In [383]:
import pandas as pd

data = {
    'Employee_Id' : ['101','102','103','104'],
    'Name' : ['John','Smith','Reg','Brik'],
    'DOB' : ['12-10-1996','13-10-1996','14-10-1996','15-10-1996'],
    'Gender':['Male','Male','Female','Female'],
    'Role': ['SDE','Data Engineer','Business Analyst','DevOps']
}

mod_data = {
    'Employee_Id' : ['101','102','103','104'],
    'Name' : ['John','Smith','Reg','Brik'],
    'DOB' : ['12-10-1996','13-11-1996','14-10-1996','15-11-1996'],
    'Gender':['Male','Female','Female','Male'],
    'Role': ['System Engineer','SDE','Business Mangager','Testing']
}

## Converting the data into Dataframe

In [384]:
dim_data= pd.DataFrame(data)
staging_data = pd.DataFrame(mod_data)

In [385]:
dim_data

,Employee_Id,Name,DOB,Gender,Role
0,101,John,12-10-1996,Male,SDE
1,102,Smith,13-10-1996,Male,Data Engineer
2,103,Reg,14-10-1996,Female,Business Analyst
3,104,Brik,15-10-1996,Female,DevOps


In [ ]:
staging_data

,Employee_Id,Name,DOB,Gender,Role
0,101,John,12-10-1996,Male,System Engineer
1,102,Smith,13-11-1996,Female,SDE
2,103,Reg,14-10-1996,Female,Business Mangager
3,104,Brik,15-11-1996,Male,Testing


## Access to User Login Credentials in SQL

In [387]:
# USE SCD;

# -- Grant user access if it doesn't already exist
# IF NOT EXISTS (
#     SELECT * FROM sys.database_principals WHERE name = 'DESKTOP-4IVU8N4\Abhinay'
# )
# BEGIN
#     CREATE USER [DESKTOP-4IVU8N4\Abhinay] FOR LOGIN [DESKTOP-4IVU8N4\Abhinay];
#     ALTER ROLE db_owner ADD MEMBER [DESKTOP-4IVU8N4\Abhinay];
# END


## Insertion of Tables

In [388]:
try:
    dim_data.to_sql("Existing_Data",con=engine,if_exists='replace',index=False)
    staging_data.to_sql("Latest_Data",con=engine,if_exists='replace',index=False)
    print("Tables are inserted")
except Exception as e:
    print("Tables are not inserted, try more you will do it!",e)

Tables are inserted


## Merge upon Existing_Data and Latest_Data

In [389]:
merged_data = pd.merge(dim_data,staging_data,on="Employee_Id",suffixes=("_old","_new"))
merged_data.head()

,Employee_Id,Name_old,DOB_old,Gender_old,Role_old,Name_new,DOB_new,Gender_new,Role_new
0,101,John,12-10-1996,Male,SDE,John,12-10-1996,Male,System Engineer
1,102,Smith,13-10-1996,Male,Data Engineer,Smith,13-11-1996,Female,SDE
2,103,Reg,14-10-1996,Female,Business Analyst,Reg,14-10-1996,Female,Business Mangager
3,104,Brik,15-10-1996,Female,DevOps,Brik,15-11-1996,Male,Testing


## Verfiying the SCD_0 Type for any Violations 

In [390]:
violations_of_scd0 = merged_data[(merged_data['DOB_old']!=merged_data['DOB_new']) | (merged_data['Gender_old']!=merged_data['Gender_new'])]

if not violations_of_scd0.empty:
    print("Violation of SCD_0, Changes in Immutuable Fields\n") 
    print(violations_of_scd0[['Employee_Id','DOB_old','DOB_new','Gender_old','Gender_new']])
else:
    print("Stay Relax, the data is free from SCD_0")

Violation of SCD_0, Changes in Immutuable Fields

  Employee_Id     DOB_old     DOB_new Gender_old Gender_new
1         102  13-10-1996  13-11-1996       Male     Female
3         104  15-10-1996  15-11-1996     Female       Male


## Creation of Log File for the violation of SCD_0 Type

In [391]:
violations_of_scd0.to_csv("scd_violations_log_file.csv",mode='a',index=False)
Cleaned_Data = merged_data[~merged_data['Employee_Id'].isin(violations_of_scd0['Employee_Id'])]
Cleaned_Data

,Employee_Id,Name_old,DOB_old,Gender_old,Role_old,Name_new,DOB_new,Gender_new,Role_new
0,101,John,12-10-1996,Male,SDE,John,12-10-1996,Male,System Engineer
2,103,Reg,14-10-1996,Female,Business Analyst,Reg,14-10-1996,Female,Business Mangager


# SCD-1

## Connect to the SQL Server

In [392]:
con = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-4IVU8N4\\MSSQLSERVER1;'
    'DATABASE=SCD;'
    'Trusted_Connection=yes;'
)

## Updating the Table Records

In [393]:
cursor = con.cursor()
insert_query="""
INSERT INTO Existing_Data(Employee_Id,Name,DOB,Gender,Role)
Values
(?,?,?,?,?)
"""
value=(105,"Jane","15-10-1996","Male","SDE")
values=(106,"Rookie","12-11-1996","Female","Data Analyst")
cursor.execute(insert_query,value)
cursor.execute(insert_query,values)
con.commit()

In [394]:
insert_query_into_latest="""
INSERT INTO Latest_Data(Employee_Id,Name,DOB,Gender,Role)
Values
(?,?,?,?,?)
"""
value_lat=(105,"Jane Smith","15-10-1996","Male","DevOps")
values_lat=(106,"Rookie Paul","12-11-1996","Female","HR")
cursor.execute(insert_query_into_latest,value_lat)
cursor.execute(insert_query_into_latest,values_lat)
con.commit()

## Reading the Table Records

In [395]:
dim_data=pd.read_sql("Select * from Existing_data",con)
dim_data

C:\Users\Abhinay\AppData\Local\Temp\ipykernel_5280\1211182508.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data=pd.read_sql("Select * from Existing_data",con)


,Employee_Id,Name,DOB,Gender,Role
0,101,John,12-10-1996,Male,SDE
1,102,Smith,13-10-1996,Male,Data Engineer
2,103,Reg,14-10-1996,Female,Business Analyst
3,104,Brik,15-10-1996,Female,DevOps
4,105,Jane,15-10-1996,Male,SDE
5,106,Rookie,12-11-1996,Female,Data Analyst


In [396]:
staging_data=pd.read_sql("Select * from Latest_Data",con)
staging_data

C:\Users\Abhinay\AppData\Local\Temp\ipykernel_5280\3706293800.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  staging_data=pd.read_sql("Select * from Latest_Data",con)


,Employee_Id,Name,DOB,Gender,Role
0,101,John,12-10-1996,Male,System Engineer
1,102,Smith,13-11-1996,Female,SDE
2,103,Reg,14-10-1996,Female,Business Mangager
3,104,Brik,15-11-1996,Male,Testing
4,105,Jane Smith,15-10-1996,Male,DevOps
5,106,Rookie Paul,12-11-1996,Female,HR


## Updating the Tables for the violation of SCD 1

In [397]:
update_query ="""
Update ed
Set
    ed.Name = ld.Name
from
    Existing_Data ed
join
    Latest_Data ld on ed.Employee_Id=ld.Employee_Id
where
    ed.Name <> ld.Name
"""
cursor.execute(update_query)
con.commit()

In [398]:
dim_data=pd.read_sql("Select * from Existing_Data",con)
dim_data

C:\Users\Abhinay\AppData\Local\Temp\ipykernel_5280\272427109.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data=pd.read_sql("Select * from Existing_Data",con)


,Employee_Id,Name,DOB,Gender,Role
0,101,John,12-10-1996,Male,SDE
1,102,Smith,13-10-1996,Male,Data Engineer
2,103,Reg,14-10-1996,Female,Business Analyst
3,104,Brik,15-10-1996,Female,DevOps
4,105,Jane Smith,15-10-1996,Male,SDE
5,106,Rookie Paul,12-11-1996,Female,Data Analyst


In [399]:
merge_data=pd.merge(dim_data,staging_data,on="Employee_Id",suffixes=('_old',"_new"))
merge_data

,Employee_Id,Name_old,DOB_old,Gender_old,Role_old,Name_new,DOB_new,Gender_new,Role_new
0,101,John,12-10-1996,Male,SDE,John,12-10-1996,Male,System Engineer
1,102,Smith,13-10-1996,Male,Data Engineer,Smith,13-11-1996,Female,SDE
2,103,Reg,14-10-1996,Female,Business Analyst,Reg,14-10-1996,Female,Business Mangager
3,104,Brik,15-10-1996,Female,DevOps,Brik,15-11-1996,Male,Testing
4,105,Jane Smith,15-10-1996,Male,SDE,Jane Smith,15-10-1996,Male,DevOps
5,106,Rookie Paul,12-11-1996,Female,Data Analyst,Rookie Paul,12-11-1996,Female,HR


## Performing the overriden process on Role Field

In [400]:
for index, row in merge_data.iterrows():
    if row['Role_old']!=row['Role_new']:
        dim_data.loc[dim_data['Employee_Id']==row['Employee_Id'],'Role']=row['Role_new']

In [401]:
dim_data

,Employee_Id,Name,DOB,Gender,Role
0,101,John,12-10-1996,Male,System Engineer
1,102,Smith,13-10-1996,Male,SDE
2,103,Reg,14-10-1996,Female,Business Mangager
3,104,Brik,15-10-1996,Female,Testing
4,105,Jane Smith,15-10-1996,Male,DevOps
5,106,Rookie Paul,12-11-1996,Female,HR


In [402]:
dim_data.to_sql("Existing_Data", con=engine, if_exists='replace', index=False)

6

In [403]:
dim_data=pd.read_sql("Select * from Existing_Data",con)
dim_data

C:\Users\Abhinay\AppData\Local\Temp\ipykernel_5280\272427109.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data=pd.read_sql("Select * from Existing_Data",con)


,Employee_Id,Name,DOB,Gender,Role
0,101,John,12-10-1996,Male,System Engineer
1,102,Smith,13-10-1996,Male,SDE
2,103,Reg,14-10-1996,Female,Business Mangager
3,104,Brik,15-10-1996,Female,Testing
4,105,Jane Smith,15-10-1996,Male,DevOps
5,106,Rookie Paul,12-11-1996,Female,HR


# SCD 2

## Pre-Defined End Date

In [404]:
safe_max_date=pd.to_datetime('2200-12-31')

## Creation of Tables through Hard Code

### Dimension Table

In [405]:
data={
    'Employee_Id':['101','102','103','104','105','106'],
    'Name' : ['John','Smith','Reg','Brik','Jane Smith','Rookie Paul'],
    'Role': ['System Engineer','SDE','Business Manager','Testing','DevOps','HR'],
    'Start_Date': pd.to_datetime(['2025-12-01','2025-11-01','2025-10-01','2025-09-01','2025-08-01','2025-07-01']),
    'End_Date':[safe_max_date]*6,
    'Active_Flag':['Yes']*6
}

### Staging Table

In [406]:
stag_data = {
    'Employee_Id':['102','106'],
    'Name' :['Smith','Rookie Paul'],
    'Role' :['HR','SDE']
}

## Conversion of Raw data into DataFrame

In [407]:
dim_status = pd.DataFrame(data)
stag_status = pd.DataFrame(stag_data)

## Sending the tables into SQL Server

In [408]:
dim_status.to_sql("Existing_Status",con=engine,if_exists='replace',index=False)
stag_status.to_sql("Latest_Status",con=engine,if_exists='replace',index=False)

2

## Extracting the Tables

In [409]:
dim_status=pd.read_sql('Select * from Existing_Status',con)
dim_status

C:\Users\Abhinay\AppData\Local\Temp\ipykernel_5280\4206214618.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_status=pd.read_sql('Select * from Existing_Status',con)


,Employee_Id,Name,Role,Start_Date,End_Date,Active_Flag
0,101,John,System Engineer,2025-12-01,2200-12-31,Yes
1,102,Smith,SDE,2025-11-01,2200-12-31,Yes
2,103,Reg,Business Manager,2025-10-01,2200-12-31,Yes
3,104,Brik,Testing,2025-09-01,2200-12-31,Yes
4,105,Jane Smith,DevOps,2025-08-01,2200-12-31,Yes
5,106,Rookie Paul,HR,2025-07-01,2200-12-31,Yes


In [410]:
stag_status=pd.read_sql('Select * from Latest_Status',con)
stag_status

C:\Users\Abhinay\AppData\Local\Temp\ipykernel_5280\474767121.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  stag_status=pd.read_sql('Select * from Latest_Status',con)


,Employee_Id,Name,Role
0,102,Smith,HR
1,106,Rookie Paul,SDE


## Verifying the SCD 2 violations

In [ ]:
from datetime import date,timedelta
safe_max_date=pd.to_datetime('2200-12-31')
today=pd.to_datetime('2025-06-24')
for index, rows in stag_status.iterrows():
    emp_id=rows['Employee_Id']
    name=rows['Name']
    role=rows['Role']

    mark=dim_status[(dim_status['Employee_Id']==emp_id) & (dim_status['Active_Flag']=='Yes')]
    
    if not mark.empty:
        current_role=mark.iloc[0]['Role']
        if role!=current_role:
            dim_status.loc[mark.index,'End_Date']=today-timedelta(days=1)
            dim_status.loc[mark.index,'Active_Flag']='No'

            new_data ={
                'Employee_Id':emp_id,
                'Name':name,
                'Role':role,
                'Start_Date':today,
                'End_Date':safe_max_date,
                'Active_Flag':'Yes'
            }
            dim_status=pd.concat([dim_status,pd.DataFrame([new_data])], ignore_index=True)

    else:
        new_data ={
                'Employee_Id':emp_id,
                'Name':name,
                'Role':role,
                'Start_Date':today,
                'End_Date':safe_max_date,
                'Active_Flag':'Yes'
            }
        dim_status=pd.concat([dim_status,pd.DataFrame([new_data])], ignore_index=True)

dim_status.sort_values(['Employee_Id'],inplace=True)
dim_status.reset_index(drop=True,inplace=True)
dim_status
                           

,Employee_Id,Name,Role,Start_Date,End_Date,Active_Flag
0,101,John,System Engineer,2025-12-01,2200-12-31,Yes
1,102,Smith,SDE,2025-11-01,2025-06-23,No
2,102,Smith,HR,2025-06-24,2200-12-31,Yes
3,103,Reg,Business Manager,2025-10-01,2200-12-31,Yes
4,104,Brik,Testing,2025-09-01,2200-12-31,Yes
5,105,Jane Smith,DevOps,2025-08-01,2200-12-31,Yes
6,106,Rookie Paul,HR,2025-07-01,2025-06-23,No
7,106,Rookie Paul,SDE,2025-06-24,2200-12-31,Yes
